In [96]:
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.sandbox.stats.multicomp import multipletests
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import statsmodels.stats.multitest as smm
from statsmodels.stats import weightstats

In [4]:
data = pd.read_csv('gene_high_throughput_sequencing.csv')
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


# Part 1

In [60]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(abs(P * (1 - P) * (1. / n1 + 1. / n2)))
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - stats.norm.cdf(z_stat)

In [61]:
normal = data[data['Diagnosis'] == 'normal']
early_neoplasia = data[data['Diagnosis'] == 'early neoplasia']
cancer = data[data['Diagnosis'] == 'cancer']

In [62]:
normal_early = []
early_cancer = []
for i in data.columns[2:]:
    z = proportions_diff_z_stat_ind(np.array(normal[i]), np.array(early_neoplasia[i]))
    normal_early.append(proportions_diff_z_test(z))
    z = proportions_diff_z_stat_ind(early_neoplasia[i], cancer[i])
    early_cancer.append(proportions_diff_z_test(z))

In [70]:
first = len(np.array(normal_early)[np.array(normal_early) < 0.05])
second = len(np.array(early_cancer)[np.array(early_cancer) < 0.05])

In [73]:
def sub_file(sub, file_name):
    with open(file_name, 'w') as file_in:
        file_in.write(str(sub))

In [74]:
sub_file((first, second), 'first_sub.txt')

In [173]:
normal_early = []
early_cancer = []
for i in data.columns[2:]:
    normal_early.append(stats.ttest_ind(normal[i], early_neoplasia[i], equal_var = False)[1])
    early_cancer.append(stats.ttest_ind(early_neoplasia[i], cancer[i], equal_var = False)[1])

In [174]:
normal_early = np.array(normal_early)
early_cancer = np.array(early_cancer)
len(normal_early[normal_early < 0.05]), len(early_cancer[early_cancer < 0.05])

(1575, 3490)

In [175]:
sub_file(len(normal_early[normal_early < 0.05]), '1.1_sub.txt')
sub_file(len(early_cancer[early_cancer < 0.05]), '1.2_sub.txt')

# Part 2

In [176]:
sum(smm.multipletests(normal_early, alpha=0.05/2, method='holm')[1] < 0.05/2)

2

In [177]:
sum(smm.multipletests(early_cancer, alpha=0.05/2, method='holm')[1] < 0.05/2)

79

In [155]:
def fold_change(T, C):
    if (T > C):
        return T/C
    else:
        return -C/T

In [178]:
fold_1 = []
fold_2 = []
for i in data.columns[2:]:
    fold_1.append(fold_change(normal[i].mean(), early_neoplasia[i].mean()))
    fold_2.append(fold_change(early_neoplasia[i].mean(), cancer[i].mean()))
fold_1 = np.array(fold_1)
fold_2 = np.array(fold_2)

In [179]:
len(fold_1[abs(fold_1) > 1.5]), len(fold_2[abs(fold_2) > 1.5])

(261, 1071)

In [180]:
for i, j in zip(smm.multipletests(normal_early, alpha=0.05/2, method='holm')[1], fold_1):
    if (i < 0.05/2):
        print(i, j)

0.012527423113260612 -1.509785482044398
0.0013383818900916007 -1.9748676656368698


In [181]:
k = 0
for i, j in zip(smm.multipletests(early_cancer, alpha=0.05/2, method='holm')[1], fold_2):
    if (i < 0.05/2 and abs(j) > 1.5):
        k+=1
print(k)

77


In [182]:
sub_file(2, '2.1_sub.txt')
sub_file(k, '2.2_sub.txt')

In [186]:
k_1 = 0
k_2 = 0
for i, j in zip(smm.multipletests(normal_early, alpha=0.05/2, method='fdr_bh')[1], fold_1):
    if (i < 0.05/2 and abs(j) > 1.5):
        k_1+=1
for i, j in zip(smm.multipletests(early_cancer, alpha=0.05/2, method='fdr_bh')[1], fold_2):
    if (i < 0.05/2 and abs(j) > 1.5):
        k_2+=1

In [188]:
sub_file(k_1, '3.1_sub.txt')
sub_file(k_2, '3.2_sub.txt')

In [187]:
k_1, k_2

(4, 524)